#1 BƯỚC 1 — Đưa nodes_parquet_all/ từ laptop lên Colab & Drive

In [1]:
# Upload file ZIP từ máy lên Colab
from google.colab import files
uploaded = files.upload()   # 👉 Chọn file nodes_parquet_all.zip từ máy

# Giải nén
!unzip -o nodes_parquet_all.zip -d /content/

# Kiểm tra cấu trúc
!find /content/nodes_parquet_all -maxdepth 2 -type f -name "*.parquet" | head -20

Saving nodes_parquet_all.zip to nodes_parquet_all.zip
Archive:  nodes_parquet_all.zip
   creating: /content/nodes_parquet_all/node_1/
  inflating: /content/nodes_parquet_all/node_1/conditions.parquet  
  inflating: /content/nodes_parquet_all/node_1/encounters.parquet  
  inflating: /content/nodes_parquet_all/node_1/medications.parquet  
  inflating: /content/nodes_parquet_all/node_1/organizations.parquet  
  inflating: /content/nodes_parquet_all/node_1/patients.parquet  
  inflating: /content/nodes_parquet_all/node_1/procedures.parquet  
   creating: /content/nodes_parquet_all/node_2/
  inflating: /content/nodes_parquet_all/node_2/conditions.parquet  
  inflating: /content/nodes_parquet_all/node_2/encounters.parquet  
  inflating: /content/nodes_parquet_all/node_2/medications.parquet  
  inflating: /content/nodes_parquet_all/node_2/organizations.parquet  
  inflating: /content/nodes_parquet_all/node_2/patients.parquet  
  inflating: /content/nodes_parquet_all/node_2/procedures.parquet 

In [2]:
import os, glob
base = '/content/nodes_parquet_all'
nodes = sorted([d for d in glob.glob(f'{base}/node_*') if os.path.isdir(d)])
print('Tổng số node:', len(nodes))

for n in nodes:
    files = glob.glob(f'{n}/*.parquet')
    print(os.path.basename(n), '→', len(files), 'files:', [os.path.basename(f) for f in files])


Tổng số node: 8
node_1 → 6 files: ['conditions.parquet', 'patients.parquet', 'medications.parquet', 'encounters.parquet', 'organizations.parquet', 'procedures.parquet']
node_2 → 6 files: ['conditions.parquet', 'patients.parquet', 'medications.parquet', 'encounters.parquet', 'organizations.parquet', 'procedures.parquet']
node_3 → 6 files: ['conditions.parquet', 'patients.parquet', 'medications.parquet', 'encounters.parquet', 'organizations.parquet', 'procedures.parquet']
node_4 → 6 files: ['conditions.parquet', 'patients.parquet', 'medications.parquet', 'encounters.parquet', 'organizations.parquet', 'procedures.parquet']
node_5 → 6 files: ['conditions.parquet', 'patients.parquet', 'medications.parquet', 'encounters.parquet', 'organizations.parquet', 'procedures.parquet']
node_6 → 6 files: ['conditions.parquet', 'patients.parquet', 'medications.parquet', 'encounters.parquet', 'organizations.parquet', 'procedures.parquet']
node_7 → 6 files: ['conditions.parquet', 'patients.parquet', 'medi

#2. BƯỚC 2 — Tạo lại hệ thống thư mục cố định

## 2.1. Tạo hai thư mục gốc đầu ra (ngay trong Colab)

In [3]:
import os, pandas as pd, numpy as np
import pyarrow.parquet as pq

# --- Cấu hình gốc ---
BASE_IN = "/content/nodes_parquet_all"
OUT_PARQUET = "/content/nodes_parquet_edit"
OUT_CSV = "/content/nodes_csv_edit"

NODES = list(range(1, 9))
TABLES = ["patients","encounters","conditions","procedures","medications","organizations"]

# --- Tạo thư mục ---
for root in [OUT_PARQUET, OUT_CSV]:
    for node in NODES:
        os.makedirs(f"{root}/node_{node}", exist_ok=True)

print("✅ Hệ thống thư mục đã tái tạo đồng bộ:")
!tree -L 2 /content | grep nodes_

# --- Hàm tiện kiểm tra schema thật ---
def check_parquet_schema(node, table, base_path=OUT_PARQUET):
    path = f"{base_path}/node_{node}/{table}.parquet"
    schema = pq.read_table(path).schema
    print(f"\n📘 Schema node_{node}/{table}.parquet")
    print(schema)

✅ Hệ thống thư mục đã tái tạo đồng bộ:
/bin/bash: line 1: tree: command not found


## 2.2. Kiểm tra lại số lượng thư mục con

In [4]:
import glob
print("Parquet edit nodes:", len(glob.glob(f"{OUT_PARQUET}/node_*")))
print("CSV edit nodes:", len(glob.glob(f"{OUT_CSV}/node_*")))

Parquet edit nodes: 8
CSV edit nodes: 8


#3. BƯỚC 3 — Kiểm tra đọc thử 1 bảng (ví dụ patients.parquet của node_1`)

## 3.1. Đọc thử bảng mẫu

In [5]:
import pandas as pd

# Đường dẫn tới file mẫu
sample_path = '/content/nodes_parquet_all/node_1/patients.parquet'

# Đọc file Parquet mẫu
df_sample = pd.read_parquet(sample_path)

# Xem 5 dòng đầu và thông tin schema
print("👀 Xem vài dòng đầu:")
display(df_sample.head())

print("\n📋 Thông tin kiểu dữ liệu:")
print(df_sample.dtypes)
print("\nTổng số dòng:", len(df_sample))

👀 Xem vài dòng đầu:


,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,...,BIRTHPLACE,ADDRESS,CITY,STATE,COUNTY,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE
0,034e9e3b-2def-4559-bb2a-7850888ae060,1983-11-14,None,999-73-5361,S99962402,X88275464X,Mr.,Milo271,Feil794,<NA>,...,Danvers Massachusetts US,422 Farrell Path Unit 69,Somerville,Massachusetts,Middlesex County,2143.0,42.360697,-71.126531,793946.01,3204.49
1,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,1996-10-18,None,999-60-7372,S99915787,X86772962X,Mr.,Gregorio366,Auer97,<NA>,...,Patras Achaea GR,1050 Lindgren Extension Apt 38,Boston,Massachusetts,Suffolk County,2135.0,42.352434,-71.028610,598763.07,3772.20
2,76982e06-f8b8-4509-9ca3-65a99c8650fe,1982-09-01,None,999-21-5604,S99957470,X55072337X,Ms.,Christal240,Brown30,<NA>,...,Bellingham Massachusetts US,1060 Hansen Overpass Suite 86,Boston,Massachusetts,Suffolk County,2118.0,42.284598,-71.134497,779464.29,11713.81
3,e4f1bd35-93ae-418b-b8cf-d39c3fbb069c,1957-02-25,None,999-66-4231,S99983425,X1708221X,Ms.,Raye931,Wunsch504,<NA>,...,Arlington Massachusetts US,585 Greenfelder Camp,Quincy,Massachusetts,Norfolk County,2169.0,42.266511,-70.956480,1551125.51,11583.44
4,bf35e4fa-ea4f-40a4-8fe6-1f2f26e0aa45,2000-11-21,None,999-87-8860,S99917788,<NA>,Ms.,Cecila397,Feil794,<NA>,...,Nahant Massachusetts US,873 Mueller Arcade Unit 96,Ashland,Massachusetts,Middlesex County,<NA>,42.213899,-71.503695,499730.64,8240.12



📋 Thông tin kiểu dữ liệu:
Id                     string[python]
BIRTHDATE                      object
DEATHDATE                      object
SSN                    string[python]
DRIVERS                string[python]
PASSPORT               string[python]
PREFIX                 string[python]
FIRST                  string[python]
LAST                   string[python]
SUFFIX                 string[python]
MAIDEN                 string[python]
MARITAL                string[python]
RACE                   string[python]
ETHNICITY              string[python]
GENDER                 string[python]
BIRTHPLACE             string[python]
ADDRESS                string[python]
CITY                   string[python]
STATE                  string[python]
COUNTY                 string[python]
ZIP                    string[python]
LAT                           float64
LON                           float64
HEALTHCARE_EXPENSES           float64
HEALTHCARE_COVERAGE           float64
dtype: object

Tổng số 

# 4. BƯỚC 4 — Chỉnh bảng patients

# 4.1 Định nghĩa hàm process cho mọi node/bảng

In [44]:
def process_table(node, tbl, edit_func):
    """Đọc bảng gốc → chỉnh → ghi ra parquet + csv"""
    df_in = pd.read_parquet(f"{BASE_IN}/node_{node}/{tbl}.parquet")
    df_edit = edit_func(df_in, node)

    pq_path = f"{OUT_PARQUET}/node_{node}/{tbl}.parquet"
    csv_path = f"{OUT_CSV}/node_{node}/{tbl}.csv"

    df_edit.to_parquet(pq_path, index=False)
    df_edit.to_csv(csv_path, index=False)
    print(f"✅ Saved node_{node}/{tbl}  ({len(df_edit)} rows)")
    return df_edit

## 4.2 – Định nghĩa lại hàm edit_patients() (đã chỉnh giữ date32)

In [55]:
import numpy as np

def edit_patients(df, node):
    df = df.copy()

    # Bảo đảm cột ngày là datetime.date (ngày-only)
    for col in ['BIRTHDATE','DEATHDATE']:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce').dt.date

    if node == 2:
        # Drop marital, ethnicity, birthplace
        for col in ['MARITAL','ETHNICITY','BIRTHPLACE']:
            if col in df.columns:
                df.drop(columns=col, inplace=True)
        # Rename ZIP -> POSTAL_CODE
        if 'ZIP' in df.columns:
            df.rename(columns={'ZIP':'POSTAL_CODE'}, inplace=True)
        # 30% birthdate NULL
        idx = df.sample(frac=0.30, random_state=2002).index
        df.loc[idx, 'BIRTHDATE'] = pd.NaT

    elif node == 3:
        # Rename gender -> sex
        if 'GENDER' in df.columns:
            df.rename(columns={'GENDER':'SEX'}, inplace=True)
        # Mask address, ssn, passport
        for col in ['ADDRESS','SSN','PASSPORT']:
            if col in df.columns:
                df[col] = 'REDACTED'
        # Add citizenship
        df['CITIZENSHIP'] = np.where(np.random.rand(len(df)) < 0.9, 'US', 'Unknown')

    elif node == 4:
        # Drop strong PII
        for col in ['SSN','DRIVERS','PASSPORT','ADDRESS','FIRST','LAST']:
            if col in df.columns:
                df.drop(columns=col, inplace=True)

    elif node == 5:
        # Add age_group, set birthdate NULL, mask address
        if 'BIRTHDATE' in df.columns:
            birth = pd.to_datetime(df['BIRTHDATE'], errors='coerce')
            age = (pd.Timestamp('today') - pd.to_datetime(birth)).dt.days / 365.25
            df['AGE_GROUP'] = pd.cut(age,
                                     bins=[0,18,35,50,65,200],
                                     labels=['child','young_adult','adult','middle_age','senior'])
            df['AGE_GROUP'] = df['AGE_GROUP'].astype('string')  # tránh categorical/dictionary
            df['BIRTHDATE'] = pd.NaT
        if 'ADDRESS' in df.columns:
            df['ADDRESS'] = 'XXXX'

    elif node == 6:
        # Standardize race/ethnicity + add ehr_vendor
        map_race = {'Caucasian':'White','African American':'Black or African American',
                    'Asian':'Asian','Hispanic':'Hispanic or Latino'}
        map_eth  = {'Hispanic':'Hispanic or Latino','Not Hispanic':'Not Hispanic or Latino'}
        if 'RACE' in df.columns:
            df['RACE'] = df['RACE'].replace(map_race)
        if 'ETHNICITY' in df.columns:
            df['ETHNICITY'] = df['ETHNICITY'].replace(map_eth)
        df['EHR_VENDOR'] = np.random.choice(['Epic','Cerner','Meditech'], len(df))
        # Add age_group nếu chưa có
        if 'AGE_GROUP' not in df.columns and 'BIRTHDATE' in df.columns:
            birth = pd.to_datetime(df['BIRTHDATE'], errors='coerce')
            age = (pd.Timestamp('today') - pd.to_datetime(birth)).dt.days / 365.25
            df['AGE_GROUP'] = pd.cut(age,
                                     bins=[0,18,35,50,65,200],
                                     labels=['child','young_adult','adult','middle_age','senior'])
            df['AGE_GROUP'] = df['AGE_GROUP'].astype('string')  # tránh categorical/dictionary

    elif node == 7:
        # gender 20% NULL + ZIP rút 3 số + lat/lon round(2)
        if 'GENDER' in df.columns:
            idx = df.sample(frac=0.20, random_state=2701).index
            df.loc[idx, 'GENDER'] = None
        if 'ZIP' in df.columns:
            df['ZIP'] = df['ZIP'].astype(str).str[:3]
        for c in ['LAT','LON']:
            if c in df.columns:
                df[c] = pd.to_numeric(df[c], errors='coerce').round(2)

    elif node == 8:
        # county 40% NULL + healthcare_coverage type mix + ethnicity 50% NULL
        if 'COUNTY' in df.columns:
            idx = df.sample(frac=0.40, random_state=2801).index
            df.loc[idx, 'COUNTY'] = None
        if 'HEALTHCARE_COVERAGE' in df.columns:
            idx = df.sample(frac=0.20, random_state=2802).index
            df.loc[idx, 'HEALTHCARE_COVERAGE'] = df.loc[idx, 'HEALTHCARE_COVERAGE'].astype(str)
            # 👇 ép toàn bộ cột sang string để ghi parquet an toàn
            df['HEALTHCARE_COVERAGE'] = df['HEALTHCARE_COVERAGE'].astype(str)
        if 'ETHNICITY' in df.columns:
            idx = df.sample(frac=0.50, random_state=2803).index
            df.loc[idx, 'ETHNICITY'] = None
        df['UPDATE_TIME'] = pd.Timestamp.now()

    return df

Chỉnh Node 1

In [8]:
# Node 1 baseline
df1 = pd.read_parquet(f"{BASE_IN}/node_1/patients.parquet")
df1.to_parquet(f"{OUT_PARQUET}/node_1/patients.parquet", index=False)
df1.to_csv(f"{OUT_CSV}/node_1/patients.csv", index=False)
print("✅ Node_1 baseline copied.")

✅ Node_1 baseline copied.


In [9]:
check_parquet_schema(1, "patients")


📘 Schema node_1/patients.parquet
Id: string
BIRTHDATE: date32[day]
DEATHDATE: date32[day]
SSN: string
DRIVERS: string
PASSPORT: string
PREFIX: string
FIRST: string
LAST: string
SUFFIX: string
MAIDEN: string
MARITAL: string
RACE: string
ETHNICITY: string
GENDER: string
BIRTHPLACE: string
ADDRESS: string
CITY: string
STATE: string
COUNTY: string
ZIP: string
LAT: double
LON: double
HEALTHCARE_EXPENSES: double
HEALTHCARE_COVERAGE: double
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 2913


Chỉnh Node 2

In [11]:
# Xử lý Node 2 bảng patients
df_node2 = process_table(2, "patients", edit_patients)

✅ Saved node_2/patients  (231 rows)


In [39]:
check_parquet_schema(2, "patients")


📘 Schema node_2/patients.parquet
Id: string
BIRTHDATE: date32[day]
DEATHDATE: date32[day]
SSN: string
DRIVERS: string
PASSPORT: string
PREFIX: string
FIRST: string
LAST: string
SUFFIX: string
MAIDEN: string
RACE: string
GENDER: string
ADDRESS: string
CITY: string
STATE: string
COUNTY: string
POSTAL_CODE: string
LAT: double
LON: double
HEALTHCARE_EXPENSES: double
HEALTHCARE_COVERAGE: double
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 2577


Chỉnh Node 3


In [13]:
# Xử lý Node 3 bảng patients
df_node3 = process_table(3, "patients", edit_patients)

✅ Saved node_3/patients  (256 rows)


In [14]:
check_parquet_schema(3, "patients")


📘 Schema node_3/patients.parquet
Id: string
BIRTHDATE: date32[day]
DEATHDATE: date32[day]
SSN: string
DRIVERS: string
PASSPORT: string
PREFIX: string
FIRST: string
LAST: string
SUFFIX: string
MAIDEN: string
MARITAL: string
RACE: string
ETHNICITY: string
SEX: string
BIRTHPLACE: string
ADDRESS: string
CITY: string
STATE: string
COUNTY: string
ZIP: string
LAT: double
LON: double
HEALTHCARE_EXPENSES: double
HEALTHCARE_COVERAGE: double
CITIZENSHIP: string
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 3029


Chỉnh Node 4

In [15]:
df_node4 = process_table(4, "patients", edit_patients)

✅ Saved node_4/patients  (367 rows)


In [16]:
check_parquet_schema(4, "patients")


📘 Schema node_4/patients.parquet
Id: string
BIRTHDATE: date32[day]
DEATHDATE: date32[day]
PREFIX: string
SUFFIX: string
MAIDEN: string
MARITAL: string
RACE: string
ETHNICITY: string
GENDER: string
BIRTHPLACE: string
CITY: string
STATE: string
COUNTY: string
ZIP: string
LAT: double
LON: double
HEALTHCARE_EXPENSES: double
HEALTHCARE_COVERAGE: double
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 2245


Chỉnh Node 5-8

In [52]:
# === 1️⃣ Xử lý 4 node còn lại ===
for node in range(5, 9):
    print(f"\n🚀 Processing Node {node} — patients")
    df_edit = process_table(node, "patients", edit_patients)
    print(f"✅ Node_{node} saved ({len(df_edit)} rows)")


🚀 Processing Node 5 — patients
✅ Saved node_5/patients  (228 rows)
✅ Node_5 saved (228 rows)

🚀 Processing Node 6 — patients
✅ Saved node_6/patients  (218 rows)
✅ Node_6 saved (218 rows)

🚀 Processing Node 7 — patients
✅ Saved node_7/patients  (281 rows)
✅ Node_7 saved (281 rows)

🚀 Processing Node 8 — patients
✅ Saved node_8/patients  (292 rows)
✅ Node_8 saved (292 rows)


/tmp/ipython-input-1995459949.py:90: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['19401.27' '4161.2' '12295.36' '3555.5599999999995' '9775.49' '7008.46'
 '10573.919999999998' '10509.499999999998' '294189.1000000001' '2949.0'
 '12613.799999999996' '2711.44' '3346.4' '22954.53' '5894.399999999999'
 '11350.77' '26102.16' '16008.839999999998' '2517.36' '8648.359999999999'
 '1599.08' '3034.0' '6946.799999999999' '8810.889999999998' '3365.6'
 '9593.09' '3148.08' '23145.03' '2661.36' '4830.049999999999'
 '15512.999999999998' '9416.07' '12597.56' '21524.47' '17603.23'
 '6233.639999999999' '78693.01999999999' '2964.8' '7960.879999999998'
 '3573.88' '1915.72' '47905.58' '2966.4' '13603.65' '7257.559999999999'
 '19290.74' '3293.89' '1679.08' '2904.84' '6895.509999999999' '1955.72'
 '8043.28' '4953.759999999999' '11095.52' '3053.12' '8614.439999999999'
 '18055.26' '2883.96']' has dtype incompatible with float64, 

In [49]:
check_parquet_schema(5, "patients")


📘 Schema node_5/patients.parquet
Id: string
BIRTHDATE: timestamp[ns]
DEATHDATE: date32[day]
SSN: string
DRIVERS: string
PASSPORT: string
PREFIX: string
FIRST: string
LAST: string
SUFFIX: string
MAIDEN: string
MARITAL: string
RACE: string
ETHNICITY: string
GENDER: string
BIRTHPLACE: string
ADDRESS: string
CITY: string
STATE: string
COUNTY: string
ZIP: string
LAT: double
LON: double
HEALTHCARE_EXPENSES: double
HEALTHCARE_COVERAGE: double
AGE_GROUP: string
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 3043


In [53]:
check_parquet_schema(6, "patients")


📘 Schema node_6/patients.parquet
Id: string
BIRTHDATE: date32[day]
DEATHDATE: date32[day]
SSN: string
DRIVERS: string
PASSPORT: string
PREFIX: string
FIRST: string
LAST: string
SUFFIX: string
MAIDEN: string
MARITAL: string
RACE: string
ETHNICITY: string
GENDER: string
BIRTHPLACE: string
ADDRESS: string
CITY: string
STATE: string
COUNTY: string
ZIP: string
LAT: double
LON: double
HEALTHCARE_EXPENSES: double
HEALTHCARE_COVERAGE: double
EHR_VENDOR: string
AGE_GROUP: string
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 3151


In [54]:
check_parquet_schema(7, "patients")


📘 Schema node_7/patients.parquet
Id: string
BIRTHDATE: date32[day]
DEATHDATE: date32[day]
SSN: string
DRIVERS: string
PASSPORT: string
PREFIX: string
FIRST: string
LAST: string
SUFFIX: string
MAIDEN: string
MARITAL: string
RACE: string
ETHNICITY: string
GENDER: string
BIRTHPLACE: string
ADDRESS: string
CITY: string
STATE: string
COUNTY: string
ZIP: string
LAT: double
LON: double
HEALTHCARE_EXPENSES: double
HEALTHCARE_COVERAGE: double
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 2913


In [56]:
check_parquet_schema(8, "patients")


📘 Schema node_8/patients.parquet
Id: string
BIRTHDATE: date32[day]
DEATHDATE: date32[day]
SSN: string
DRIVERS: string
PASSPORT: string
PREFIX: string
FIRST: string
LAST: string
SUFFIX: string
MAIDEN: string
MARITAL: string
RACE: string
ETHNICITY: string
GENDER: string
BIRTHPLACE: string
ADDRESS: string
CITY: string
STATE: string
COUNTY: string
ZIP: string
LAT: double
LON: double
HEALTHCARE_EXPENSES: double
HEALTHCARE_COVERAGE: string
UPDATE_TIME: timestamp[us]
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 3043


## 4.3 – Định nghĩa hàm edit_encounters()

In [68]:
def edit_encounters(df, node):
    df = df.copy()
    # Chuẩn datetime cho START/STOP trước khi xử lý
    for col in ['START','STOP']:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')

    if node == 1:
        # Baseline: giữ nguyên
        return df

    elif node == 2:
        # 1️⃣ payer 20% NULL
        if 'PAYER' in df.columns:
            idx = df.sample(frac=0.20, random_state=2201).index
            df.loc[idx, 'PAYER'] = None
        # 2️⃣ rename base_encounter_cost → enc_cost
        if 'BASE_ENCOUNTER_COST' in df.columns:
            df.rename(columns={'BASE_ENCOUNTER_COST':'ENC_COST'}, inplace=True)
        # 3️⃣ drop payer_coverage
        if 'PAYER_COVERAGE' in df.columns:
            df.drop(columns=['PAYER_COVERAGE'], inplace=True)
        return df

    elif node == 3:
        # 1️⃣ format START/STOP thành chuỗi "MM/DD/YYYY HH:MM"
        for col in ['START','STOP']:
            if col in df.columns:
                df[col] = df[col].dt.strftime('%m/%d/%Y %H:%M').astype('string')
        # 2️⃣ rename ORGANIZATION → hospital_id
        if 'ORGANIZATION' in df.columns:
            df.rename(columns={'ORGANIZATION':'HOSPITAL_ID'}, inplace=True)
        return df

    elif node == 4:
        # 1. base_encounter_cost → string “100.0 USD”
        if 'BASE_ENCOUNTER_COST' in df.columns:
            df['BASE_ENCOUNTER_COST'] = df['BASE_ENCOUNTER_COST'].apply(lambda x: f"{float(x):.1f} USD")
        # 2. add visit_type (“emergency”, “routine”, “follow-up”)
        df['VISIT_TYPE'] = np.random.choice(['emergency','routine','follow-up'], len(df))
        return df

    elif node == 5:
        # 1️⃣ START 10% NULL
        if 'START' in df.columns:
            idx = df.sample(frac=0.10, random_state=2505).index
            df.loc[idx, 'START'] = pd.NaT
        # 2️⃣ DROP hoàn toàn REASONCODE & REASONDESCRIPTION
        df = df.drop(columns=[c for c in ['REASONCODE', 'REASONDESCRIPTION'] if c in df.columns])
        return df

    elif node == 6:
        # # 1️⃣ encounterclass standardized (INP/AMB/EMR)
        # if 'ENCOUNTERCLASS' in df.columns:
        #     df['ENCOUNTERCLASS'] = np.random.choice(['INP','AMB','EMR'], len(df))
        # 2️⃣ add ehr_vendor (“Cerner”, “Epic”, “Meditech”)
        df['EHR_VENDOR'] = np.random.choice(['Cerner','Epic','Meditech'], len(df))
        return df

    elif node == 7:
        # 1️⃣ ORGANIZATION 5% NULL
        if 'ORGANIZATION' in df.columns:
            idx = df.sample(frac=0.05, random_state=2707).index
            df.loc[idx, 'ORGANIZATION'] = None
        # 2️⃣ total_claim_cost ±10% random
        if 'TOTAL_CLAIM_COST' in df.columns:
            df['TOTAL_CLAIM_COST'] = df['TOTAL_CLAIM_COST'] * (1 + np.random.uniform(-0.1, 0.1, len(df)))
        return df

    elif node == 8:
        # 1. add update_time (timestamp export)
        df['UPDATE_TIME'] = pd.Timestamp.now()
        return df

    return df


Chạy node 1-8

In [70]:
for node in range(1, 9):
    print(f"\n🚀 Processing Node {node} — encounters")
    _ = process_table(node, "encounters", edit_encounters)


🚀 Processing Node 1 — encounters
✅ Saved node_1/encounters  (8496 rows)

🚀 Processing Node 2 — encounters
✅ Saved node_2/encounters  (6662 rows)

🚀 Processing Node 3 — encounters
✅ Saved node_3/encounters  (5948 rows)

🚀 Processing Node 4 — encounters
✅ Saved node_4/encounters  (9431 rows)

🚀 Processing Node 5 — encounters
✅ Saved node_5/encounters  (8471 rows)

🚀 Processing Node 6 — encounters
✅ Saved node_6/encounters  (3882 rows)

🚀 Processing Node 7 — encounters
✅ Saved node_7/encounters  (4740 rows)

🚀 Processing Node 8 — encounters
✅ Saved node_8/encounters  (5716 rows)


In [74]:
check_parquet_schema(1, "encounters")


📘 Schema node_1/encounters.parquet
Id: string
START: timestamp[ns, tz=UTC]
STOP: timestamp[ns, tz=UTC]
PATIENT: string
ORGANIZATION: string
PROVIDER: string
PAYER: string
ENCOUNTERCLASS: string
CODE: string
DESCRIPTION: string
BASE_ENCOUNTER_COST: double
TOTAL_CLAIM_COST: double
PAYER_COVERAGE: double
REASONCODE: string
REASONDESCRIPTION: string
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 1926


In [63]:
check_parquet_schema(2, "encounters")


📘 Schema node_2/encounters.parquet
Id: string
START: timestamp[ns, tz=UTC]
STOP: timestamp[ns, tz=UTC]
PATIENT: string
ORGANIZATION: string
PROVIDER: string
PAYER: string
ENCOUNTERCLASS: string
CODE: string
DESCRIPTION: string
ENC_COST: double
TOTAL_CLAIM_COST: double
REASONCODE: string
REASONDESCRIPTION: string
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 1775


In [64]:
check_parquet_schema(3, "encounters")


📘 Schema node_3/encounters.parquet
Id: string
START: string
STOP: string
PATIENT: string
HOSPITAL_ID: string
PROVIDER: string
PAYER: string
ENCOUNTERCLASS: string
CODE: string
DESCRIPTION: string
BASE_ENCOUNTER_COST: double
TOTAL_CLAIM_COST: double
PAYER_COVERAGE: double
REASONCODE: string
REASONDESCRIPTION: string
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 1872


In [65]:
check_parquet_schema(4, "encounters")


📘 Schema node_4/encounters.parquet
Id: string
START: timestamp[ns, tz=UTC]
STOP: timestamp[ns, tz=UTC]
PATIENT: string
ORGANIZATION: string
PROVIDER: string
PAYER: string
ENCOUNTERCLASS: string
CODE: string
DESCRIPTION: string
BASE_ENCOUNTER_COST: string
TOTAL_CLAIM_COST: double
PAYER_COVERAGE: double
REASONCODE: string
REASONDESCRIPTION: string
VISIT_TYPE: string
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 2045


In [66]:
check_parquet_schema(5, "encounters")


📘 Schema node_5/encounters.parquet
Id: string
START: timestamp[ns, tz=UTC]
STOP: timestamp[ns, tz=UTC]
PATIENT: string
ORGANIZATION: string
PROVIDER: string
PAYER: string
ENCOUNTERCLASS: string
CODE: string
DESCRIPTION: string
BASE_ENCOUNTER_COST: double
TOTAL_CLAIM_COST: double
PAYER_COVERAGE: double
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 1672


In [71]:
check_parquet_schema(6, "encounters")


📘 Schema node_6/encounters.parquet
Id: string
START: timestamp[ns, tz=UTC]
STOP: timestamp[ns, tz=UTC]
PATIENT: string
ORGANIZATION: string
PROVIDER: string
PAYER: string
ENCOUNTERCLASS: string
CODE: string
DESCRIPTION: string
BASE_ENCOUNTER_COST: double
TOTAL_CLAIM_COST: double
PAYER_COVERAGE: double
REASONCODE: string
REASONDESCRIPTION: string
EHR_VENDOR: string
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 2046


In [72]:
check_parquet_schema(7, "encounters")


📘 Schema node_7/encounters.parquet
Id: string
START: timestamp[ns, tz=UTC]
STOP: timestamp[ns, tz=UTC]
PATIENT: string
ORGANIZATION: string
PROVIDER: string
PAYER: string
ENCOUNTERCLASS: string
CODE: string
DESCRIPTION: string
BASE_ENCOUNTER_COST: double
TOTAL_CLAIM_COST: double
PAYER_COVERAGE: double
REASONCODE: string
REASONDESCRIPTION: string
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 1926


In [73]:
check_parquet_schema(8, "encounters")


📘 Schema node_8/encounters.parquet
Id: string
START: timestamp[ns, tz=UTC]
STOP: timestamp[ns, tz=UTC]
PATIENT: string
ORGANIZATION: string
PROVIDER: string
PAYER: string
ENCOUNTERCLASS: string
CODE: string
DESCRIPTION: string
BASE_ENCOUNTER_COST: double
TOTAL_CLAIM_COST: double
PAYER_COVERAGE: double
REASONCODE: string
REASONDESCRIPTION: string
UPDATE_TIME: timestamp[us]
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 2057


## 4.4 – Định nghĩa hàm edit_conditions()

In [77]:
def edit_conditions(df, node):
    df = df.copy()

    # Chuẩn hoá kiểu ngày (start, stop)
    for col in ['START','STOP']:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce').dt.date

    if node == 1:
        return df  # baseline

    elif node == 2:
        # 1 drop STOP
        if 'STOP' in df.columns:
            df.drop(columns=['STOP'], inplace=True)
        # (2) Rename code → diagnosis_code
        if 'CODE' in df.columns:
            df.rename(columns={'CODE':'DIAGNOSIS_CODE'}, inplace=True)
        return df

    elif node == 3:
        # (1) Mix ICD-9 / ICD-10 (một nửa dòng đổi pattern)
        if 'CODE' in df.columns:
            mask = np.random.rand(len(df)) < 0.5
            df.loc[mask, 'CODE'] = df.loc[mask, 'CODE'].astype(str).str.replace('^I', '250.', regex=True)
        # (2) Description = "REDACTED" cho bệnh HIV hoặc mental
        if 'DESCRIPTION' in df.columns:
            df.loc[df['DESCRIPTION'].str.contains('HIV|mental', case=False, na=False), 'DESCRIPTION'] = 'REDACTED'
        return df

    elif node == 4:
        # (1) 15 % START = NULL
        if 'START' in df.columns:
            idx = df.sample(frac=0.15, random_state=2404).index
            df.loc[idx, 'START'] = pd.NaT
        # (2) Multi-code “I10, E11”
        if 'CODE' in df.columns:
            df['CODE'] = df['CODE'].astype(str).replace('^I10$', 'I10, E11', regex=True)
        return df

    elif node == 5:
        # (1) Thêm cột SEVERITY (“mild”/“moderate”/“severe”)
        df['SEVERITY'] = np.random.choice(['mild','moderate','severe'], len(df))
        return df

    elif node == 6:
        # (1) DESCRIPTION Title Case
        if 'DESCRIPTION' in df.columns:
            df['DESCRIPTION'] = df['DESCRIPTION'].astype(str).str.title()
        # (2) Thêm EHR_VENDOR = “Epic”
        df['EHR_VENDOR'] = 'Epic'
        return df

    elif node == 7:
        # (1) Drop sensitive codes (bệnh reproductive, HIV, etc.)
        if 'DESCRIPTION' in df.columns:
            drop_idx = df[df['DESCRIPTION'].str.contains(
                'reproductive|HIV|pregnan|fertil|sexual', case=False, na=False)].index
            df = df.drop(drop_idx)
        return df

    elif node == 8:
        # (1) 20 % DESCRIPTION = NULL
        if 'DESCRIPTION' in df.columns:
            idx = df.sample(frac=0.20, random_state=2808).index
            df.loc[idx, 'DESCRIPTION'] = None
        # (2) Thêm UPDATE_TIME (timestamp export)
        df['UPDATE_TIME'] = pd.Timestamp.now()
        return df

    return df


Chạy node 1-8

In [78]:
for node in range(1, 9):
    print(f"\n🚀 Processing Node {node} — conditions")
    _ = process_table(node, "conditions", edit_conditions)


🚀 Processing Node 1 — conditions
✅ Saved node_1/conditions  (1344 rows)

🚀 Processing Node 2 — conditions
✅ Saved node_2/conditions  (1107 rows)

🚀 Processing Node 3 — conditions
✅ Saved node_3/conditions  (1050 rows)

🚀 Processing Node 4 — conditions
✅ Saved node_4/conditions  (1692 rows)

🚀 Processing Node 5 — conditions
✅ Saved node_5/conditions  (904 rows)

🚀 Processing Node 6 — conditions
✅ Saved node_6/conditions  (665 rows)

🚀 Processing Node 7 — conditions
✅ Saved node_7/conditions  (732 rows)

🚀 Processing Node 8 — conditions
✅ Saved node_8/conditions  (837 rows)


In [79]:
# Kiểm tra schema thật
for node in range(1, 9):
    check_parquet_schema(node, "conditions")


📘 Schema node_1/conditions.parquet
START: date32[day]
STOP: date32[day]
PATIENT: string
ENCOUNTER: string
CODE: string
DESCRIPTION: string
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 749

📘 Schema node_2/conditions.parquet
START: date32[day]
PATIENT: string
ENCOUNTER: string
DIAGNOSIS_CODE: string
DESCRIPTION: string
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 664

📘 Schema node_3/conditions.parquet
START: date32[day]
STOP: date32[day]
PATIENT: string
ENCOUNTER: string
CODE: string
DESCRIPTION: string
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 749

📘 Schema node_4/conditions.parquet
START: date32[day]
STOP: date32[day]
PATIENT: string
ENCOUNTER: string
CODE: string
DESCRIPTION: string
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 749

📘 Schema node_5/conditions.parquet


## 4.5 – Định nghĩa hàm edit_procedures()

In [82]:
def edit_procedures(df, node):
    df = df.copy()

    # Chuẩn hóa cột ngày
    if "DATE" in df.columns:
        df["DATE"] = pd.to_datetime(df["DATE"], errors="coerce")

    if node == 1:
        return df  # baseline

    elif node == 2:
        # rename base_cost → procedure_cost
        if "BASE_COST" in df.columns:
            df.rename(columns={"BASE_COST": "PROCEDURE_COST"}, inplace=True)
        # drop reasoncode / reasondescription
        for col in ["REASONCODE", "REASONDESCRIPTION"]:
            if col in df.columns:
                df.drop(columns=col, inplace=True)
        return df

    elif node == 3:
        # mix CPT / ICD-10
        if "CODE" in df.columns:
            mask = np.random.rand(len(df)) < 0.5
            df.loc[mask, "CODE"] = df.loc[mask, "CODE"].astype(str).str.replace(
                "^I", "992", regex=True
            )  # CPT code
        # REDACTED reproductive
        if "DESCRIPTION" in df.columns:
            df.loc[
                df["DESCRIPTION"].str.contains("reproductive", case=False, na=False),
                "DESCRIPTION",
            ] = "REDACTED"
        return df

    elif node == 4:
        # 10 % DATE = NULL
        if "DATE" in df.columns:
            idx = df.sample(frac=0.10, random_state=2404).index
            df.loc[idx, "DATE"] = pd.NaT
        # base_cost string “500 USD”
        if "BASE_COST" in df.columns:
            df["BASE_COST"] = df["BASE_COST"].apply(lambda x: f"{float(x):.1f} USD")
        return df

    elif node == 5:
        # Add severity (“minor” / “major”)
        df["SEVERITY"] = np.random.choice(["minor", "major"], len(df))
        return df

    elif node == 6:
        # DESCRIPTION Title Case
        if "DESCRIPTION" in df.columns:
            df["DESCRIPTION"] = df["DESCRIPTION"].astype(str).str.title()
        # Add ehr_vendor (“Epic”, “Cerner”, “Meditech”)
        df["EHR_VENDOR"] = np.random.choice(
            ["Epic", "Cerner", "Meditech"], len(df)
        )
        return df

    elif node == 7:
        # Drop sensitive codes
        if "DESCRIPTION" in df.columns:
            drop_idx = df[
                df["DESCRIPTION"].str.contains(
                    "reproductive|HIV|pregnan|fertil|sexual",
                    case=False,
                    na=False,
                )
            ].index
            df = df.drop(drop_idx)

        return df

    elif node == 8:
        # Add update_time (audit timestamp)
        df["UPDATE_TIME"] = pd.Timestamp.now()
        return df

    return df


Chạy node 1-8

In [83]:
for node in range(1, 9):
    print(f"\n🚀 Processing Node {node} — procedures")
    _ = process_table(node, "procedures", edit_procedures)



🚀 Processing Node 1 — procedures
✅ Saved node_1/procedures  (6187 rows)

🚀 Processing Node 2 — procedures
✅ Saved node_2/procedures  (4409 rows)

🚀 Processing Node 3 — procedures
✅ Saved node_3/procedures  (3968 rows)

🚀 Processing Node 4 — procedures
✅ Saved node_4/procedures  (6793 rows)

🚀 Processing Node 5 — procedures
✅ Saved node_5/procedures  (5883 rows)

🚀 Processing Node 6 — procedures
✅ Saved node_6/procedures  (2147 rows)

🚀 Processing Node 7 — procedures
✅ Saved node_7/procedures  (2655 rows)

🚀 Processing Node 8 — procedures
✅ Saved node_8/procedures  (2862 rows)


In [85]:
# Kiểm tra schema thật
for node in range(1, 9):
    check_parquet_schema(node, "procedures")


📘 Schema node_1/procedures.parquet
DATE: timestamp[ns, tz=UTC]
PATIENT: string
ENCOUNTER: string
CODE: string
DESCRIPTION: string
BASE_COST: double
REASONCODE: string
REASONDESCRIPTION: string
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 1044

📘 Schema node_2/procedures.parquet
DATE: timestamp[ns, tz=UTC]
PATIENT: string
ENCOUNTER: string
CODE: string
DESCRIPTION: string
PROCEDURE_COST: double
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 800

📘 Schema node_3/procedures.parquet
DATE: timestamp[ns, tz=UTC]
PATIENT: string
ENCOUNTER: string
CODE: string
DESCRIPTION: string
BASE_COST: double
REASONCODE: string
REASONDESCRIPTION: string
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 1044

📘 Schema node_4/procedures.parquet
DATE: timestamp[ns, tz=UTC]
PATIENT: string
ENCOUNTER: string
CODE: string
DESCRIPTION: string
BASE_COST: strin

## 4.6 – Định nghĩa hàm edit_medications()

In [95]:
def edit_medications(df, node):
    df = df.copy()

    # Chuẩn hoá datetime cho START / STOP
    for col in ['START', 'STOP']:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')

    if node == 1:
        return df  # baseline

    elif node == 2:
        # (1) rename base_cost → drug_cost
        if 'BASE_COST' in df.columns:
            df.rename(columns={'BASE_COST': 'DRUG_COST'}, inplace=True)
        # (2) drop payer_coverage
        if 'PAYER_COVERAGE' in df.columns:
            df.drop(columns=['PAYER_COVERAGE'], inplace=True)
        return df

    elif node == 3:
        # (1) REDACTED (HIV/mental)
        if 'DESCRIPTION' in df.columns:
            df.loc[df['DESCRIPTION'].str.contains('HIV|mental', case=False, na=False), 'DESCRIPTION'] = 'REDACTED'
        # (2) base_cost → string "XX USD"
        if 'BASE_COST' in df.columns:
            df['BASE_COST'] = df['BASE_COST'].apply(lambda x: f"{float(x):.1f} USD")
        return df

    elif node == 4:
        # (1) payer 25 % NULL
        if 'PAYER' in df.columns:
            idx = df.sample(frac=0.25, random_state=2404).index
            df.loc[idx, 'PAYER'] = None
        # (2) thêm cột form (“tablet”, “injection”, “capsule”)
        df['FORM'] = np.random.choice(['tablet','injection','capsule'], len(df))
        return df

    elif node == 5:
        # (1) drop reasoncode / reasondescription
        for col in ['REASONCODE', 'REASONDESCRIPTION']:
            if col in df.columns:
                df.drop(columns=col, inplace=True)
        return df

    elif node == 6:
        # (1) code → RxNorm short form (cắt 3 ký tự đầu)
        if 'CODE' in df.columns:
            df['CODE'] = df['CODE'].astype(str).str[:5]
        # (2) add ehr_vendor (“Epic”, “Cerner”, “Meditech”)
        df['EHR_VENDOR'] = np.random.choice(['Epic','Cerner','Meditech'], len(df))
        return df

    elif node == 7:
        # (1) payer 10 % NULL
        if 'PAYER' in df.columns:
            idx = df.sample(frac=0.10, random_state=2707).index
            df.loc[idx, 'PAYER'] = None
        # (2) dispenses → float type (đảm bảo float)
        if 'DISPENSES' in df.columns:
            df['DISPENSES'] = pd.to_numeric(df['DISPENSES'], errors='coerce').astype(float)
        # (3) totalcost rounded (1 decimal)
        if 'TOTALCOST' in df.columns:
            df['TOTALCOST'] = pd.to_numeric(df['TOTALCOST'], errors='coerce').round(1)
        return df

    elif node == 8:
        # (1) add update_time (timestamp audit)
        df['UPDATE_TIME'] = pd.Timestamp.now()
        return df

    return df


Chạy node 1-8

In [96]:
for node in range(1, 9):
    print(f"\n🚀 Processing Node {node} — medications")
    _ = process_table(node, "medications", edit_medications)



🚀 Processing Node 1 — medications
✅ Saved node_1/medications  (6233 rows)

🚀 Processing Node 2 — medications
✅ Saved node_2/medications  (5960 rows)

🚀 Processing Node 3 — medications
✅ Saved node_3/medications  (5319 rows)

🚀 Processing Node 4 — medications
✅ Saved node_4/medications  (7659 rows)

🚀 Processing Node 5 — medications
✅ Saved node_5/medications  (8091 rows)

🚀 Processing Node 6 — medications
✅ Saved node_6/medications  (3235 rows)

🚀 Processing Node 7 — medications
✅ Saved node_7/medications  (2669 rows)

🚀 Processing Node 8 — medications
✅ Saved node_8/medications  (3823 rows)


In [97]:
# Kiểm tra schema thật
for node in range(1, 9):
    check_parquet_schema(node, "medications")


📘 Schema node_1/medications.parquet
START: timestamp[ns, tz=UTC]
STOP: timestamp[ns, tz=UTC]
PATIENT: string
PAYER: string
ENCOUNTER: string
CODE: string
DESCRIPTION: string
BASE_COST: double
PAYER_COVERAGE: double
DISPENSES: int64
TOTALCOST: double
REASONCODE: string
REASONDESCRIPTION: string
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 1653

📘 Schema node_2/medications.parquet
START: timestamp[ns, tz=UTC]
STOP: timestamp[ns, tz=UTC]
PATIENT: string
PAYER: string
ENCOUNTER: string
CODE: string
DESCRIPTION: string
DRUG_COST: double
DISPENSES: int64
TOTALCOST: double
REASONCODE: string
REASONDESCRIPTION: string
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 1524

📘 Schema node_3/medications.parquet
START: timestamp[ns, tz=UTC]
STOP: timestamp[ns, tz=UTC]
PATIENT: string
PAYER: string
ENCOUNTER: string
CODE: string
DESCRIPTION: string
BASE_COST: string
PAYER_COVERAGE: double
DISPENS

## 4.7 – Định nghĩa hàm edit_organizations()

In [102]:
def edit_organizations(df, node):
    df = df.copy()

    if node == 1:
        return df  # baseline

    elif node == 2:
        # (1) rename revenue → annual_revenue
        if 'REVENUE' in df.columns:
            df.rename(columns={'REVENUE': 'ANNUAL_REVENUE'}, inplace=True)
        # (2) drop utilization
        if 'UTILIZATION' in df.columns:
            df.drop(columns=['UTILIZATION'], inplace=True)
        return df

    elif node == 3:
        # (1) revenue string “1.2M USD”
        if 'REVENUE' in df.columns:
            df['REVENUE'] = (df['REVENUE'] / 1_000).round(1).astype(str) + 'K USD'
        # (2) 40 % lat/lon NULL
        for c in ['LAT', 'LON']:
            if c in df.columns:
                idx = df.sample(frac=0.40, random_state=2303).index
                df.loc[idx, c] = None
        return df

    elif node == 4:
        # (1) ZIP → format ZIP+4 (ví dụ 02139-1234)
        if 'ZIP' in df.columns:
            df['ZIP'] = df['ZIP'].astype(str).str[:5] + '-' + np.random.randint(1000,9999,len(df)).astype(str)
        # (2) Thêm license_id (“HOSP-xxxx”)
        df['LICENSE_ID'] = ['HOSP-' + str(x).zfill(4) for x in range(1, len(df)+1)]
        return df

    elif node == 5:
        # (1) ±15 % REVENUE
        if 'REVENUE' in df.columns:
            df['REVENUE'] = df['REVENUE'] * (1 + np.random.uniform(-0.15,0.15,len(df)))

        return df

    elif node == 6:
        # (1) Thêm EHR_VENDOR (Epic/Cerner/Meditech)
        df['EHR_VENDOR'] = np.random.choice(['Epic','Cerner','Meditech'], len(df))
        return df

    elif node == 7:
        # (1) ADDRESS dropped
        if 'ADDRESS' in df.columns:
            df.drop(columns=['ADDRESS'], inplace=True)
        # (2) PHONE dropped
        if 'PHONE' in df.columns:
            df.drop(columns=['PHONE'], inplace=True)
        # (3) LAT/LON rounded (1 decimal)
        for c in ['LAT','LON']:
            if c in df.columns:
                df[c] = df[c].round(1)
        return df

    elif node == 8:
        # (1) drop REVENUE
        if 'REVENUE' in df.columns:
            df.drop(columns=['REVENUE'], inplace=True)
        # (2) thêm update_time (audit timestamp)
        df['UPDATE_TIME'] = pd.Timestamp.now()
        return df

    return df

CHạy node 1-8

In [103]:
for node in range(1, 9):
    print(f"\n🚀 Processing Node {node} — organizations")
    _ = process_table(node, "organizations", edit_organizations)


🚀 Processing Node 1 — organizations
✅ Saved node_1/organizations  (139 rows)

🚀 Processing Node 2 — organizations
✅ Saved node_2/organizations  (130 rows)

🚀 Processing Node 3 — organizations
✅ Saved node_3/organizations  (137 rows)

🚀 Processing Node 4 — organizations
✅ Saved node_4/organizations  (136 rows)

🚀 Processing Node 5 — organizations
✅ Saved node_5/organizations  (139 rows)

🚀 Processing Node 6 — organizations
✅ Saved node_6/organizations  (140 rows)

🚀 Processing Node 7 — organizations
✅ Saved node_7/organizations  (139 rows)

🚀 Processing Node 8 — organizations
✅ Saved node_8/organizations  (159 rows)


In [104]:
# Kiểm tra schema thật
for node in range(1, 9):
    check_parquet_schema(node, "organizations")


📘 Schema node_1/organizations.parquet
Id: string
NAME: string
ADDRESS: string
CITY: string
STATE: string
ZIP: string
LAT: double
LON: double
PHONE: string
REVENUE: double
UTILIZATION: double
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 1287

📘 Schema node_2/organizations.parquet
Id: string
NAME: string
ADDRESS: string
CITY: string
STATE: string
ZIP: string
LAT: double
LON: double
PHONE: string
ANNUAL_REVENUE: double
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 1178

📘 Schema node_3/organizations.parquet
Id: string
NAME: string
ADDRESS: string
CITY: string
STATE: string
ZIP: string
LAT: double
LON: double
PHONE: string
REVENUE: string
UTILIZATION: double
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 1286

📘 Schema node_4/organizations.parquet
Id: string
NAME: string
ADDRESS: string
CITY: string
STATE: string
ZIP: string
LAT: do

#5 DOWNLOAD ZIP NODE CSV/PARQUET

In [105]:
!zip -r -q /content/nodes_csv_edit.zip /content/nodes_csv_edit
print("✅ Đã nén xong nodes_csv_edit.zip")

✅ Đã nén xong nodes_csv_edit.zip


In [106]:
!zip -r -q /content/nodes_parquet_edit.zip /content/nodes_parquet_edit
print("✅ Đã nén xong nodes_parquet_edit.zip")

✅ Đã nén xong nodes_parquet_edit.zip


In [107]:
from google.colab import files

files.download("/content/nodes_csv_edit.zip")
files.download("/content/nodes_parquet_edit.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>